# Data Compilation



This code consolidates seperate .csv data files sourced from the internet into one data frame, drops certain observations and fills NAs.

In [1]:
import numpy as np
import pandas as pd

##############################################################################
#Define help fuunctions to load data
##############################################################################

#This is a dictioary that maps selected country names to JHU format
country_dictionary = {
                      #United States
                      "United States of America" : "US", 
                      "U.S." : "US",
                      "United States" : "US",
                      "USA" : "US",

                      #United Kingdom
                      "UK" : "United Kingdom",
                      
                      #Others
                      "Republic of Korea" : "Korea, South",
                      "Russian Federation" : " Russia",
                      "Iran (Islamic Republic of)" : "Iran",
                      "Bolivia (Plurinational State of)":"Bolivia",
                      "Republic of North Macedonia" : "North Macedonia",
                      "Côte d'Ivoire" : "Cote d'Ivoire",
                      "Congo" : "Congo (Brazzaville)",
                      "Taiwan" : "Taiwan*",
                      "UAE": "United Arab Emirates"}

#define a function that renames country names to JHU format
def rename_countries(x, y, country_dictionary = None):
    
    #delete possible spaces " " at the end of a string, if applicable
    a = y
    
    #apply the dictionary-based remap, if applicable
    if not(country_dictionary is None):
        a = a.replace(country_dictionary)
    
    #loop through the list of country names and assign matches
    for x_str in x:
        bool_list = np.vectorize(lambda z: z.find(x_str)>=0)(y)
        
        if (np.sum(bool_list) > 1):
            if np.sum(np.vectorize(lambda z: x_str == z)(a)) != 1:
                print("Error: Multiple Matches")
                break
                #return
        else:
            a[bool_list] = x_str
    return a


##############################################################################
#Load data into a big data frame
##############################################################################
    
#create list of files
import os
path = "./Raw Data"
file_list = list(filter(lambda x: ('.csv' in x) and not('JHU_recent_mortality_by_country' in x), os.listdir(path)))

#load target variable
jhu = pd.read_csv(path+"/JHU_recent_mortality_by_country.csv", 
                  converters = {"Case-Fatality": lambda x: float(x.strip("%"))/100})

#loop through list of files in directory
for file_name in file_list:
    
    #load current file
    who = pd.read_csv(path+"/"+file_name, encoding = "latin-1").iloc[:,:2]
    who.rename(columns={who.columns[0]: "Country" }, inplace=True)
    
    #delete possible spaces " " at the end and beginning of a string, if applicable
    who["Country"] = np.vectorize(lambda z: z.rstrip().lstrip())(who["Country"])
    
    #remap for values encoded as "<0.1", if applicable
    if type(who.iloc[0,1]) == str:
        who.iloc[:,1] = who.iloc[:,1].replace({"<0.1" : "0.05"}).astype(float)

    #apply country remapping function
    who["Country"] = rename_countries(jhu["Country"], who["Country"], country_dictionary)
    
    #append to master data base
    jhu = jhu.merge(who, on ="Country", how = "left")

del(who, file_name, file_list, country_dictionary, path)


##############################################################################
#Drop small countries
##############################################################################

def drop_countries(data = jhu, quantile_threshold = 0.4):
    
    data["Population"] = data["Population"].fillna(0)
    quantile = np.quantile(data["Population"], quantile_threshold)
    data = data.loc[ data["Population"] > quantile ]
    return data

jhu = drop_countries(data = jhu, quantile_threshold = 0.4)

#drop population column
jhu = jhu.drop("Population",axis = 1)

##############################################################################
#Handle NAs
##############################################################################

#function that drops every country that has less than half of  the data available
def drop_NA(data = jhu, country_thresh = 0.5, var_thresh = 0.7):
    
    #delete countries with too many NAs
    bool_vec = (1 - np.apply_along_axis(lambda x: np.sum(np.isnan(x)), 1, data.iloc[:,1:])/data.iloc[:,1:].shape[1])>=country_thresh
    a = data.loc[bool_vec]
    
    #delete variables with too many NAs
    bool_vec = (1 - np.apply_along_axis(lambda x: np.sum(np.isnan(x)), 0, data.iloc[:,1:])/data.iloc[:,1:].shape[0])>=var_thresh
    a = a.loc[:, np.append(True,bool_vec)]

    return a


#drop NAs for countries that has less than half of  the data available
jhu = drop_NA(country_thresh = 0.9, var_thresh = 0.8)


def fill_NA(input_data = jhu, dev = 0.5, statistics = "median", method = "OLS"):
    
    from sklearn.linear_model import HuberRegressor, LinearRegression
    from sklearn.preprocessing import StandardScaler
    
    data = np.array(input_data.iloc[:,2:])
    
    #list of variables in data with NAs
    var_list = np.arange(data.shape[1])[np.apply_along_axis(lambda x: np.sum(np.isnan(x)), 0, data) >= 1]
    
    if method == "OLS":
        reg_method = LinearRegression(fit_intercept = False)
    else:
        reg_method = HuberRegressor(fit_intercept = False)
    
    if statistics == "mean":
        average = np.mean
        disp = np.std
    else:
        from scipy import stats
        average = np.median
        disp = stats.median_absolute_deviation
        
        
    #loop through variables containing NAs
    for i in var_list:
        
        #indices corresponding to test observation indices
        test_indx = np.where( np.isnan(data[:,i]) )[0]
        
        #(preliminary) indices corresponding to train observation indices
        train_indx = np.where( ~(np.isnan(data[:,i])))[0]
        
        #indices corresponding to variables
        var_indx = np.where(np.apply_along_axis(lambda x: np.sum(np.isnan(x)) == 0, 0, data[test_indx,:]))[0]
        
        #test set of X-variables and y-variables
        X_test = data[test_indx,:][:,var_indx] 
        indx_temp = np.where(np.apply_along_axis(lambda x: np.sum(np.isnan(x)) == 0, 1, data[train_indx,:][:, var_indx]))[0]
        
        #define train set for OLS regression
        X_train = data[train_indx,:][:, var_indx][indx_temp,:]
        y_train = data[train_indx,i][indx_temp]
    
        #standardize training and testing data
        scaler = StandardScaler(with_mean=True, with_std=True).fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        scaler = StandardScaler(with_mean=True, with_std=True).fit(y_train.reshape(-1,1))
        y_train = scaler.transform(y_train.reshape(-1,1))
        
        #perform OLS regression
        reg = reg_method.fit(X_train, y_train)
        reg_pred = scaler.inverse_transform(reg.predict(X_test)).ravel()
        y_train = scaler.inverse_transform(y_train.reshape(-1,1)).ravel()
        
        #prune results if they deviate too much
        lower, upper = average(y_train)-dev*disp(y_train), average(y_train)+dev*disp(y_train)
        reg_pred[lower > reg_pred] = lower
        reg_pred[upper < reg_pred] = upper
        
        #fill NAs
        data[test_indx, i] = reg_pred

    input_data.iloc[:,2:] = data
    return input_data

#fill NAs
jhu = fill_NA(input_data = jhu, dev = 0.5, statistics = "median", method = "OLS")


##############################################################################
#Export data
##############################################################################

jhu.to_csv("raw_data.csv", index = False)